In [0]:
# Mounting Directory for Yelp Data Intake
storageAccount = "gen10datafund2205"
storageContainer = "group1-marketing-analytics"
mount_point = "/mnt/group1/datain"
clientSecret = "-ZS8Q~NwOKfwEpVOg3Teb1pPtxDbz616XjlXLbuU"
clientid = "2ca50102-5717-4373-b796-39d06568588d"
    
    
configs2 = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs2)

/mnt/group1/datain has been unmounted.
Out[1]: True

In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)

In [0]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupyter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "Group1-Marketing-Analytics"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"


#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,# this will create a new consumer group on each invocation.
    'group.id': str(1),
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': True,
    'error_cb': error_cb,
})

c.subscribe([confluentTopicName])

In [0]:
aString = {}

kafkaListDictionaries = []

while True:
    try:
        msg = c.poll(timeout=15)
        print(msg)
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            aString=json.loads('{}'.format(msg.value().decode('utf-8')))
            #aString['timestamp'] = msg.timestamp()[1]
            kafkaListDictionaries.append(aString)
            c.commit(asynchronous=False)
    except Exception as e:
        print(e)

<cimpl.Message object at 0x7f4f357d1ec0>
<cimpl.Message object at 0x7f4f292b61c0>
<cimpl.Message object at 0x7f4f357d1ec0>
<cimpl.Message object at 0x7f4f292b61c0>
<cimpl.Message object at 0x7f4f357d1ec0>
<cimpl.Message object at 0x7f4f292b65c0>
<cimpl.Message object at 0x7f4f357d1ec0>
<cimpl.Message object at 0x7f4f292b65c0>
<cimpl.Message object at 0x7f4f357d1ec0>
<cimpl.Message object at 0x7f4f292b65c0>
<cimpl.Message object at 0x7f4f357d1ec0>
<cimpl.Message object at 0x7f4f292b65c0>
<cimpl.Message object at 0x7f4f357d1ec0>
None

In [0]:
database = "Group1-Marketing-Analytics"
table = "dbo.State"
user = "Group1"
password  = "D9k8n7z6g%mar!keting"
server = "gen10-data-fundamentals-22-05-sql-server.database.windows.net"

In [0]:
df = spark.createDataFrame(kafkaListDictionaries)

In [0]:
df.coalesce(1).write.mode('overwrite').csv(mount_point+'/state')

In [0]:
try:
    df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
        .mode("append") \
        .option("dbtable", table) \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .save()
except:
    print('An Error Occured.')

An Error Occured.